In [ ]:
!pip install -q peft transformers datasets evaluate peft -q accelerate
!pip install huggingface_hub
!pip install bert_score

In [2]:
from datasets import load_dataset, concatenate_datasets
import evaluate
import numpy as np
from transformers.data.metrics.squad_metrics import compute_predictions_logits
from transformers import (
    AutoModelForSeq2SeqLM,
    AutoModelForQuestionAnswering,
    AutoTokenizer,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
)
from peft import (
    get_peft_config,
    get_peft_model,
    get_peft_model_state_dict,
    set_peft_model_state_dict,
    PeftType,
    PromptEncoderConfig,
)
import torch
from huggingface_hub import notebook_login
from transformers import AutoModelForCausalLM, AutoTokenizer, default_data_collator, get_linear_schedule_with_warmup
from peft import get_peft_config, get_peft_model, PromptTuningInit, PromptTuningConfig, TaskType, PeftType
import torch
from datasets import load_dataset
import os
from torch.utils.data import DataLoader
from tqdm import tqdm

In [ ]:
notebook_login()

In [4]:
dataset = load_dataset('minh21/cpgQA-v1.0-unique-context-for-flan-t5')
device = "cuda"
model_name='google/flan-t5-large'
lr = 3e-5

In [5]:
train_dataset = dataset['train']
test_dataset = dataset['test']

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True) #Convert text to vector space
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

# Zero shot

Test the tokenizer encoding and decoding a simple sentence:

In [44]:
bf_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [45]:
def generate_predictions(example):
    question = example["question"]
    context = example["context"]
    id = example['id']
    prompt = f"""
    Read this and answer the question. If the question is unanswerable, "
    say \"unanswerable\".\n\n{context}\n\n{question}",
    """

    # Input constructed prompt instead of the dialogue.
    inputs = tokenizer(prompt, return_tensors='pt')
    output = tokenizer.decode(
        bf_model.generate(
            inputs["input_ids"],
            max_new_tokens=50,
        )[0],
        skip_special_tokens=True
    )
    answer = {
        'prediction_text': output,
        'no_answer_probability': 0,
        'id': str(id)
    }
    return answer

predictions = []
for i, index in enumerate(test_dataset):
    # predictions.append(index)
    predictions.append(generate_predictions(index))


predictions.__len__()


144

In [ ]:
predictions

In [46]:
from evaluate import load
squad_metric = load("squad_v2")
results = squad_metric.compute(predictions=predictions, references=references)

results

{'exact': 59.72222222222222,
 'f1': 82.08038451752687,
 'total': 144,
 'HasAns_exact': 59.72222222222222,
 'HasAns_f1': 82.08038451752687,
 'HasAns_total': 144,
 'best_exact': 59.72222222222222,
 'best_exact_thresh': 0.0,
 'best_f1': 82.08038451752687,
 'best_f1_thresh': 0.0}

# One shot

In [ ]:
def generate_predictions_one_shot(example, example2):
    question_train = example2["question"]
    context_train = example2["context"]
    answer_text_train = example2["answer_text"]

    question = example["question"]
    context = example["context"]
    id = example['id']
    prompt = f"""
    Using this example:
    [EXAMPLE]Read this and answer the question. If the question is unanswerable, "
    say \"unanswerable\".\n\n{context_train}\n\n{question_train}", "{answer_text_train}"

    To do
    Read this and answer the question. If the question is unanswerable, "
    say \"unanswerable\".\n\n{context}\n\n{question}",
    """

    # Input constructed prompt instead of the dialogue.
    inputs = tokenizer(prompt, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"],
            max_new_tokens=50,
        )[0],
        skip_special_tokens=True
    )
    answer = {
        'prediction_text': output,
        'no_answer_probability': 0,
        'id': str(id)
    }
    return answer

predictions_one_shot = []
for i, index in enumerate(test_dataset):
    if(i>0):
      predictions_one_shot.append(generate_predictions_one_shot(test_dataset[i], test_dataset[i-1]))
    else:
      predictions_one_shot.append(generate_predictions_one_shot(test_dataset[i], test_dataset[test_dataset.__len__()-1]))


predictions_one_shot

In [ ]:
from evaluate import load
squad_metric = load("squad_v2")
results = squad_metric.compute(predictions=predictions_one_shot, references=references)

results

{'exact': 51.81818181818182,
 'f1': 75.3573430317572,
 'total': 110,
 'HasAns_exact': 51.81818181818182,
 'HasAns_f1': 75.3573430317572,
 'HasAns_total': 110,
 'best_exact': 51.81818181818182,
 'best_exact_thresh': 0.0,
 'best_f1': 75.3573430317572,
 'best_f1_thresh': 0.0}

In [ ]:
def generate_predictions_one_shot(example, example2):
    question_train = example2["question"]
    context_train = example2["context"]
    answer_text_train = example2["answer_text"]

    question = example["question"]
    context = example["context"]
    id = example['id']
    prompt = f"""
    Using this example:
    [CONTEXT]: {context_train}\n
    [QUESTION]: {question_train} \n
    [ANSWER]: {answer_text_train} \n

    [CONTEXT]: {context}\n
    [QUESTION]: {question} \n
    [ANSWER]:
    """

    # Input constructed prompt instead of the dialogue.
    inputs = tokenizer(prompt, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"],
            max_new_tokens=50,
        )[0],
        skip_special_tokens=True
    )
    answer = {
        'prediction_text': output,
        'no_answer_probability': 0,
        'id': str(id)
    }
    return answer

predictions_one_shot = []
for i, index in enumerate(test_dataset):
    if(i>0):
      predictions_one_shot.append(generate_predictions_one_shot(test_dataset[i], test_dataset[i-1]))
    else:
      predictions_one_shot.append(generate_predictions_one_shot(test_dataset[i], test_dataset[test_dataset.__len__()-1]))


predictions_one_shot




In [ ]:
from evaluate import load
squad_metric = load("squad_v2")
results = squad_metric.compute(predictions=predictions_one_shot, references=references)

results

{'exact': 55.45454545454545,
 'f1': 79.78866711083056,
 'total': 110,
 'HasAns_exact': 55.45454545454545,
 'HasAns_f1': 79.78866711083056,
 'HasAns_total': 110,
 'best_exact': 55.45454545454545,
 'best_exact_thresh': 0.0,
 'best_f1': 79.78866711083056,
 'best_f1_thresh': 0.0}

# Several shot

In [ ]:
def generate_predictions_several_shots(example, example2, example3, example4):
    question_train_2 = example2["question"]
    context_train_2 = example2["context"]
    answer_text_train_2 = example2["answer_text"]

    question_train_3 = example3["question"]
    context_train_3 = example3["context"]
    answer_text_train_3 = example3["answer_text"]

    question_train_4 = example4["question"]
    context_train_4 = example4["context"]
    answer_text_train_4 = example4["answer_text"]

    question = example["question"]
    context = example["context"]
    id = example['id']
    prompt = f"""
    Using this example:
    [CONTEXT]: {context_train_2}\n
    [QUESTION]: {question_train_2}\n
    [ANSWER]: {answer_text_train_2}\n

    [CONTEXT]: {context_train_3}\n
    [QUESTION]: {question_train_3}\n
    [ANSWER]: {answer_text_train_3}\n

    [CONTEXT]: {context_train_4}\n
    [QUESTION]: {question_train_4}\n
    [ANSWER]: {answer_text_train_4}\n

    [CONTEXT]: {context}\n
    [QUESTION]: {question} \n
    [ANSWER]:
    """

    # Input constructed prompt instead of the dialogue.
    inputs = tokenizer(prompt, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"],
            max_new_tokens=50,
        )[0],
        skip_special_tokens=True
    )
    answer = {
        'prediction_text': output,
        'no_answer_probability': 0,
        'id': str(id)
    }
    return answer

predictions_several_shots = []
for i, index in enumerate(test_dataset):
    if(i>3):
      predictions_several_shots.append(generate_predictions_several_shots(test_dataset[i], test_dataset[i-1], test_dataset[i-2], test_dataset[i-3]))
    else:
      predictions_several_shots.append(generate_predictions_several_shots(test_dataset[i], test_dataset[test_dataset.__len__()-i-1], test_dataset[test_dataset.__len__()-i-2], test_dataset[test_dataset.__len__()-i-3] ))


predictions_several_shots




In [ ]:
from evaluate import load
squad_metric = load("squad_v2")
results = squad_metric.compute(predictions=predictions_several_shots, references=references)

results

{'exact': 55.45454545454545,
 'f1': 79.22830556919165,
 'total': 110,
 'HasAns_exact': 55.45454545454545,
 'HasAns_f1': 79.22830556919165,
 'HasAns_total': 110,
 'best_exact': 55.45454545454545,
 'best_exact_thresh': 0.0,
 'best_f1': 79.22830556919165,
 'best_f1_thresh': 0.0}

# Prompt Tuning

In [18]:
peft_config = PromptTuningConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    prompt_tuning_init=PromptTuningInit.TEXT,
    num_virtual_tokens=20,
    prompt_tuning_init_text="Answer this question using the provided context",
    tokenizer_name_or_path=model_name,
)

In [21]:
model_max_length = tokenizer.model_max_length

In [22]:
# target_max_length = max([len(tokenizer(data['context'])["input_ids"]) for data in train_dataset])

maxx = 0
dat = []
for data in train_dataset:
  text = data['context'] + "\n" + data['question']
  l = tokenizer(text)["input_ids"].__len__()
  if l>=512:
    dat.append(data)
  if l >= maxx:
    maxx = l
dat
maxx

481

In [23]:
def tokenize_function(data):
    context = data['context']
    question = data['question']
    answer = data['answer_text']
    id = data['id']
    input = f"""
    Read this and answer the question. If the question is unanswerable, "
    say \"unanswerable\".\n\n{context}\n\n{question}",
    """
    model_inputs = tokenizer(
        input,
        padding='max_length',
        max_length=512,
        truncation=True
    )

    labels = tokenizer(answer)

    model_inputs["labels"] = labels["input_ids"]
    labels['id'] = id
    return model_inputs

In [ ]:
processed_datasets = {
    'train': train_dataset.map(
    tokenize_function,
    batched=False,
    num_proc=1,
    remove_columns=train_dataset.column_names,
    load_from_cache_file=False,
    desc="Running tokenizer on train_dataset",
    ),
    'test': test_dataset.map(
    tokenize_function,
    batched=False,
    num_proc=1,
    remove_columns=train_dataset.column_names,
    load_from_cache_file=False,
    desc="Running tokenizer on test_dataset",
  )
}

In [26]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
model = get_peft_model(model, peft_config)
label_pad_token_id = -100
# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)

def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(model))

trainable model parameters: 40960
all model parameters: 783191040
percentage of trainable model parameters: 0.01%
trainable params: 40,960 || all params: 783,191,040 || trainable%: 0.005229886184601908
None


In [29]:
training_args = Seq2SeqTrainingArguments(
    evaluation_strategy="epoch",
    learning_rate=lr,
    do_eval=False,
    output_dir="./flan-t5-large-prompt-tuning-cpgQA",
    per_device_train_batch_size=2,
    num_train_epochs=5,
    weight_decay=0.01,
    lr_scheduler_type="linear",
    push_to_hub=True,
    save_strategy="no",
    logging_steps=200,
)

In [30]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=processed_datasets["train"],
    eval_dataset=processed_datasets["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    # optimizers=[optimizer, lr_scheduler],
)


In [31]:
trainer.train()

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,0.192000,0.136314
2,0.240200,0.136312
3,0.224900,0.136309
4,0.224300,0.136309
5,0.224300,0.136308


TrainOutput(global_step=2150, training_loss=0.22789144649062046, metrics={'train_runtime': 1800.7615, 'train_samples_per_second': 2.388, 'train_steps_per_second': 1.194, 'total_flos': 9910502188646400.0, 'train_loss': 0.22789144649062046, 'epoch': 5.0})

In [37]:
trainer.push_to_hub()

'https://huggingface.co/minh21/flan-t5-large-prompt-tuning-cpgQA/tree/main/'

# Load model from hub

In [ ]:
# Load adapters from hub
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

peft_model_id = "minh21/flan-t5-large-prompt-tuning-cpgQA"
config = PeftConfig.from_pretrained(peft_model_id)

model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

model = PeftModel.from_pretrained(model, peft_model_id)

print("Peft model loaded")

# Evaluation model

In [38]:
model.to('cuda')
predictions_for_squad = []
predictions_for_squad_v2 = []
predictions_for_bert_score = []
references_for_bert_score = []
for data in test_dataset:
    context = data['context']
    question = data['question']
    answer = data['answer_text']
    id = data['id']
    input = f"""
    "{context}\nTry to answer this question if possible (otherwise reply "
        "\"unanswerable\"): {question}"
    """
    model_inputs = tokenizer(
        input,
        padding='max_length',
        max_length=512,
        truncation=True,
        return_tensors='pt'
    ).to(torch.device("cuda"))


    model_output = tokenizer.decode(
        model.generate(
            input_ids=model_inputs["input_ids"],
            attention_mask=model_inputs["attention_mask"],
        )[0],
        skip_special_tokens=True
    )

    predictions_for_squad.append({
          'prediction_text': model_output,
          'id': str(id),
    })

    predictions_for_squad_v2.append({
          'prediction_text': model_output,
          'no_answer_probability': 0,
          'id': str(id),
    })

    predictions_for_bert_score.append(model_output)
    references_for_bert_score.append(answer)
    # predictions.extend(predicted_texts)

In [39]:
references_for_squad_v2 = [
    {
        "answers": {"answer_start": [ds["answer_start"]], "text": [ds["answer_text"]]},
        "id": str(ds["id"]),
    }
    for id, ds in enumerate(test_dataset)
]

In [42]:
from evaluate import load
results = dict()
squad_metric = load("squad_v2")
results["squad_v2"] = squad_metric.compute(predictions=predictions_for_squad_v2, references=references_for_squad_v2)

squad_metric = load("squad")
results["squad"] = squad_metric.compute(predictions=predictions_for_squad, references=references_for_squad_v2)

bleu_metrics = load("bleu")
prediction = ["hello there general kenobi","foo bar foobar"]
reference = [
["hello there general kenobi"],
["foo bar foobar"]
]
results["bleu"] = bleu_metrics.compute(predictions=predictions_for_bert_score, references=references_for_bert_score)

bertscore_metric = load("bertscore")
berscore = bertscore_metric.compute(
    predictions=predictions_for_bert_score, references=references_for_bert_score, lang="en"
)

results["bertscore"] = {
    "precision": sum(berscore["precision"]) / len(berscore["precision"]),
    "recall": sum(berscore["recall"]) / len(berscore["recall"]),
    "f1": sum(berscore["f1"]) / len(berscore["f1"]),
}
results

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'squad_v2': {'exact': 47.916666666666664,
  'f1': 75.94572464040252,
  'total': 144,
  'HasAns_exact': 47.916666666666664,
  'HasAns_f1': 75.94572464040252,
  'HasAns_total': 144,
  'best_exact': 47.916666666666664,
  'best_exact_thresh': 0.0,
  'best_f1': 75.94572464040252,
  'best_f1_thresh': 0.0},
 'squad': {'exact_match': 47.916666666666664, 'f1': 75.94572464040252},
 'bleu': {'bleu': 0.4510316021376081,
  'precisions': [0.9239704329461457,
   0.9028642590286425,
   0.886094674556213,
   0.8792184724689165],
  'brevity_penalty': 0.5023340369535013,
  'length_ratio': 0.592245153220763,
  'translation_length': 947,
  'reference_length': 1599},
 'bertscore': {'precision': 0.9606295786798,
  'recall': 0.9368742116623454,
  'f1': 0.9481206101675829}}